In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from sklearn.preprocessing import LabelEncoder
import os
import joblib

In [ ]:
# Defind the correct order of columns
correct_order = ['name', 'year', 'selling_price', 'km_driven', 'fuel', 'transmission', 'owner', 'mileage', 'engine', 'max_power', 'seats']

# 1. Giới thiệu

Vấn đề, tầm quan trọng, mục tiêu dự án.

Giới thiệu tổng quan về dự án, tập trung vào việc dự đoán giá xe ở Ấn Độ và phân tích các yếu tố ảnh hưởng.

- Trình bày vấn đề: Dự đoán giá xe dựa trên các thuộc tính như tuổi xe, loại nhiên liệu, km đã đi, v.v., và phân cụm để hiểu phân khúc thị trường.

- Giải thích tầm quan trọng: Thị trường xe cũ ở Ấn Độ phát triển mạnh (khoảng 3,4 triệu xe giao dịch mỗi năm theo IJERT), ảnh hưởng bởi chính sách (BS-VI), và nhu cầu cá nhân hóa sau COVID-19.
- Đặt mục tiêu: Xây dựng mô hình Random Forest để dự đoán giá xe chính xác.
Sử dụng K-Means để phân cụm xe thành các nhóm (ví dụ: xe giá rẻ, xe cao cấp).
Khám phá các yếu tố chính ảnh hưởng đến giá và phân khúc.

# 2. Hiểu dữ liệu

- Mô tả: Nguồn dữ liệu, mô tả các thuộc tính, số lượng bản ghi.
- Scope: Mô tả tập dữ liệu bạn đã có, đảm bảo phù hợp với thị trường Ấn Độ và đủ thông tin cho cả dự đoán giá và phân cụm.
- Quy trình:
    1. Nêu nguồn dữ liệu: Ví dụ, từ Kaggle, công ty ô tô, hoặc tự thu thập.
    2. Mô tả các thuộc tính: Liệt kê tất cả cột (giá, nhãn hiệu, năm sản xuất, loại nhiên liệu, km đã đi, loại truyền động, v.v.) và ý nghĩa của chúng.
    3. Thống kê cơ bản: Số lượng bản ghi (rows), số cột (columns), kiểu dữ liệu (numeric, categorical).
- Công việc cụ thể:
    - Dùng pandas trong Python để đọc dữ liệu (df.head(), df.info(), df.describe()).
    - Ghi chú các đặc điểm ban đầu: Ví dụ, "Dữ liệu gồm 10.000 xe, với giá từ 50.000 INR đến 5 triệu INR".
    - Kiểm tra xem dữ liệu có đại diện cho thị trường Ấn Độ không (nhiều nhãn hiệu như Maruti Suzuki, Hyundai không?).

#### Load dữ liệu

In [ ]:
# Load Data
raw_df1 = pd.read_csv('raw_csv/cardekho.csv')
raw_df2 = pd.read_csv('raw_csv/train.csv')
raw_df3 = pd.read_csv('raw_csv/processes2.csv')


#### Hiển thị, mô tả các thuộc tính, số lượng bản ghi

In [ ]:
print(raw_df1.shape)
raw_df1.head()

In [ ]:
print(raw_df2.shape)
raw_df2.head()

In [ ]:
print(raw_df3.shape)
raw_df3.head()

# 3. Phân tích Khám phá Dữ liệu (EDA)

- Mô tả: Thống kê, biểu đồ, mối quan hệ giữa giá và các yếu tố.
- Scope: Hiểu dữ liệu thô, phát hiện xu hướng, vấn đề (giá trị thiếu, ngoại lai), và chuẩn bị cho phân cụm K-Means.
- Quy trình:
    1. Thống kê mô tả: Tính trung bình, trung vị, độ lệch chuẩn của giá, tuổi xe, km đã đi.
    2. Trực quan hóa:
        -  Biểu đồ phân phối giá (histogram) để xem phân khúc giá phổ biến.
        -  Biểu đồ phân tán (scatter plot) giữa giá và tuổi xe/km đã đi.
        -  Biểu đồ hộp (box plot) để phát hiện ngoại lai (ví dụ: giá xe bất thường > 10 triệu INR).
        -  Biểu đồ thanh (bar plot) để so sánh giá trung bình theo loại nhiên liệu/nhãn hiệu.
    3. Phân tích mối quan hệ: Tính tương quan (correlation) giữa giá và các biến số (dùng df.corr()).
- Công việc cụ thể:
    - Sử dụng thư viện như matplotlib, seaborn để vẽ biểu đồ.
    - Ghi nhận phát hiện: Ví dụ, "Xe dầu diesel có giá trung bình cao hơn xe xăng 15%".
    - Chuẩn bị cho K-Means: Xem các biến như giá, km đã đi, tuổi xe có thể phân cụm xe thành các nhóm không.

**Dựa trên những thuộc tính đã khám phá từ 2., xử lý sơ bằng các chuyển kiểu dữ liệu về cho đúng để thuận tiện cho việc vẽ**

## Chuẩn hóa tên cột của tập dữ liệu, kiểu dữ liệu và đơn vị

In [ ]:
raw_df1.info()

In [ ]:
raw_df1 = raw_df1[(raw_df1['fuel']!='CNG') & (raw_df1['fuel']!='LPG') & (raw_df1['owner']!='Test Drive Car')]
raw_df1['owner'] = raw_df1['owner'].str.replace(' Owner','',regex=True)
raw_df1['name'] = raw_df1['name'].str.split().str[0]# .str.title()
raw_df1['selling_price'] = raw_df1['selling_price'] / 100000.0

In [ ]:
# Dictionary ánh xạ từ tên cột cũ sang tên cột mới
column_mapping_df1 = {
    "mileage(km/ltr/kg)": "mileage"
}
raw_df1 = raw_df1.drop(columns=['seller_type'])
raw_df1.rename(columns=column_mapping_df1, inplace=True)
print(raw_df1.columns)
raw_df1.head()

In [ ]:
raw_df2.info()

In [ ]:
print(raw_df2['Owner_Type'].unique())
raw_df2 = raw_df2[raw_df2['Fuel_Type']!='Electric']
raw_df2['Engine'] = raw_df2['Engine'].str.extract('(\d+)')  # Lấy chỉ số trong chuỗi
raw_df2['Power'] = raw_df2['Power'].str.extract('(\d+)')  # Lấy chỉ số trong chuỗi
raw_df2['Mileage'] = raw_df2['Mileage'].str.extract('(\d+)')  # Lấy chỉ số trong chuỗi
raw_df2 = raw_df2[~raw_df2['Mileage'].str.contains('km/kg', na=False)]
raw_df2['Name'] = raw_df2['Name'].str.split(' ').str[0]
raw_df2.head()


In [ ]:

column_mapping_df2 = {
        "Year": "year",
        "Kilometers_Driven": "km_driven",
        "Fuel_Type": "fuel",
        "Transmission": "transmission",
        "Mileage": "mileage",
        "Engine": "engine",
        "Power": "max_power",
        "Seats": "seats",
        "Price": "selling_price",
        "Name": "name",
        "Owner_Type": "owner"
    }
raw_df2 = raw_df2.drop(columns=['New_Price', 'Location','Unnamed: 0'])
raw_df2.rename(columns=column_mapping_df2, inplace=True)
raw_df2 = raw_df2[correct_order]
raw_df2.info()
raw_df2.head()

In [ ]:
raw_df3.info()

In [ ]:
# Đọc dữ liệu từ file csv
raw_df3 = pd.read_csv('raw_csv/processes2.csv')
raw_df3 = raw_df3[(raw_df3['fuel']!='CNG') & (raw_df3['fuel']!='LPG') & (raw_df3['owner']!='Test Drive Car')]
raw_df3['owner'] = raw_df3['owner'].str.replace(' Owner','',regex=True)
raw_df3['selling_price'] = raw_df3['selling_price'] / 100000.0
print(raw_df3['fuel'].unique())
print(raw_df3['owner'].unique())
raw_df3.head()

In [ ]:
column_mapping_df3 = {
        "max_power (in bph)": "max_power",
        "Mileage": "mileage",
        "Engine (CC)": "engine"
    }
raw_df3 = raw_df3.drop(columns=['seller_type', 'Mileage Unit', 'Unnamed: 0'])
raw_df3.rename(columns=column_mapping_df3, inplace=True)
raw_df3 = raw_df3[correct_order]

In [ ]:
def plot_histograms(df, columns, figsize=(15, 12), bins_method='sqrt'):
    """
    Vẽ histogram cho các cột liên tục trong DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu.
        columns (list): Danh sách tên các cột cần vẽ histogram.
        figsize (tuple): Kích thước của figure (mặc định là (15, 12)).
        bins_method (str): Phương pháp tính bin cho numpy.histogram_bin_edges() (mặc định là 'sqrt').

    Returns:
        None (Hiển thị biểu đồ).
    """
    plt.figure(figsize=figsize)

    for index, col in enumerate(columns):
        plt.subplot(2, 2, index + 1)
        
        # Chuyển cột về kiểu số, bỏ giá trị không hợp lệ
        data = pd.to_numeric(df[col], errors='coerce').dropna()
        
        # Tính toán bin edges bằng phương pháp chỉ định
        bin_edges = np.histogram_bin_edges(data, bins=bins_method)
        num_bins = len(bin_edges) - 1  # Số bin thực tế
        
        # Vẽ histogram với số bin tính toán được
        sns.histplot(data=data, bins=num_bins, color=sns.color_palette('pastel')[index], kde=True)
        
        plt.title(col.replace('_', ' ').capitalize(), fontsize=14, pad=10)
        plt.xlabel(col.replace('_', ' ').capitalize(), fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tick_params(axis='both', labelsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
continuous_raw_columns = ['km_driven', 'mileage', 'engine', 'max_power']

In [ ]:
raw_df1.info()
raw_df1.describe()

In [ ]:
plot_histograms(raw_df1, continuous_raw_columns)

In [ ]:
raw_df2.info()
raw_df2.describe()

In [ ]:
plot_histograms(raw_df2, continuous_raw_columns)

In [ ]:
raw_df3.info()
raw_df3.head()

In [ ]:
plot_histograms(raw_df3, continuous_raw_columns)

# 4. Tiền xử lý dữ liệu

In [ ]:
# Merge 3 datasets
merged_df = pd.concat([raw_df1, raw_df2, raw_df3], ignore_index=True)

# Export to CSV file
saved_file_path = 'raw_csv/merged_dataset.csv'
merged_df.to_csv(saved_file_path, index=False)

# Print message
print(f"Merged and saved to '{saved_file_path}'")

## Load merged data

In [ ]:
raw_df = pd.read_csv(saved_file_path)
print(raw_df.shape)
raw_df.head()

In [ ]:
raw_df.info()
raw_df.describe()

## Explore data analysis

In [ ]:
# Take copy from data to make processes on
preprocessed_df = deepcopy(raw_df)

# Show all types of columns in the data
preprocessed_df.dtypes

In [ ]:
# Check about none values in data to decide if we will make data cleaning or not
preprocessed_df.isnull().sum()

## Handling missing values

In [ ]:
# Handle missing values of float columns that are mileage, engine, seats
column_float_imputed = ['mileage', 'engine', 'max_power']
preprocessed_df[column_float_imputed] = preprocessed_df[column_float_imputed].fillna(preprocessed_df[column_float_imputed].mean())

# Handle missing values of seats column 
preprocessed_df['seats'] = preprocessed_df['seats'].fillna(preprocessed_df['seats'].mode()[0])

# Check if there is any missing values in the dataset
print(preprocessed_df.isnull().sum())


## Handling textual columns

In [ ]:
# Show all textual columns in the dataset
textual_columns = preprocessed_df.select_dtypes(include = ['object']).columns
preprocessed_df[textual_columns]


## Overview

In [ ]:
# Define category columns
categorical_columns = ['name', 'fuel', 'transmission', 'owner', 'seats', 'year']
threshold = 5.0

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()  # Làm phẳng mảng axes để dễ dàng lặp qua

def plot_pie_chart(df, column, ax):

    value_counts = df[column].value_counts()
    percentages = value_counts / value_counts.sum() * 100
    
    mask = percentages < threshold
    if mask.any():
        other_percentage = percentages[mask].sum()
        percentages = percentages[~mask]
        percentages['Other'] = other_percentage
    
    colors = plt.cm.jet(np.linspace(0, 1, len(percentages)))
    ax.pie(percentages, labels=percentages.index, autopct='%1.1f%%', 
           startangle=90, shadow=True, explode=[0.05] * len(percentages),
           colors=colors)
    ax.axis('equal')
    ax.set_title(f'{column}', fontsize=14)

for i, column in enumerate(categorical_columns):
    if column in preprocessed_df.columns and i < len(axes):
        plot_pie_chart(preprocessed_df, column, axes[i])

plt.tight_layout()
plt.show()


In [ ]:
plot_histograms(preprocessed_df, continuous_raw_columns)

# # Define continuos columns
# continuous_columns = ['km_driven', 'mileage', 'engine', 'max_power']

# plt.figure(figsize=(15, 12))

# for index, col in enumerate(continuous_columns):
#     plt.subplot(2, 2, index + 1)
    
#     sns.histplot(data=preprocessed_df[col], bins=50, 
#                  color=sns.color_palette('pastel')[index], kde=True)
    
#     plt.title(col.replace('_', ' ').capitalize(), fontsize=14, pad=10)
    
#     plt.xlabel(col.replace('_', ' ').capitalize(), fontsize=12)
#     plt.ylabel('Frequency', fontsize=12)
    
#     plt.grid(True, linestyle='--', alpha=0.7)
    
#     plt.tick_params(axis='both', labelsize=10)


# plt.tight_layout()
# plt.show()

## Label encoding textual columns

In [ ]:
# Dictionary to store the encoders
label_encoders = {}

# Fit and transform each textual column, saving the encoders
for col in textual_columns:
    label_encoder = LabelEncoder()
    preprocessed_df[col] = label_encoder.fit_transform(preprocessed_df[col])
    label_encoders[col] = label_encoder

preprocessed_df.head()

In [ ]:
# Check about if there is any extra textual columns
preprocessed_df.select_dtypes(include=['object']).columns

## Correlation matrix

In [ ]:
# Compute the correlation matrix
correlation_matrix = preprocessed_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='Blues', 
            annot_kws={"size": 10}, linewidths=0.5, fmt=".2f")
plt.title('Correlation Matrix', fontsize=20)
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Remove the selling_price 
corr_with_price = correlation_matrix['selling_price'].drop('selling_price').sort_values(ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(x=corr_with_price.index, y=corr_with_price.values, 
            hue=corr_with_price.index, palette='Blues_d')

plt.title('Correlation of Features with Selling Price', fontsize=16, pad=20)

plt.xlabel('Features', fontsize=12)
plt.ylabel('Correlation Coefficient', fontsize=12)

plt.xticks(rotation=45, ha='right', fontsize=10)

plt.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## Handling outliers

In [ ]:
# Remove selling_price column
column_skewed = preprocessed_df.columns.drop('selling_price')

# Compute skewness for each column
skewness = preprocessed_df.drop(columns=['selling_price']).skew()
print(skewness)

plt.figure(figsize=(15, 7))
sns.barplot(x=column_skewed, y=skewness, hue=column_skewed, palette='coolwarm')

plt.title('Skewness of Features in Car Dataset', fontsize=16, pad=20)

plt.xlabel('Features', fontsize=12)
plt.ylabel('Skewness', fontsize=12)

plt.axhline(y=0, color='black', linestyle='--', linewidth=1)

plt.xticks(rotation=45, ha='right', fontsize=10)

plt.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Handle skewness of km driven columns by taking the log function for it
preprocessed_df['km_driven'] = np.log(preprocessed_df['km_driven'] + 1)

In [ ]:
# Find skewness for km driven column after we handled it
preprocessed_df['km_driven'].skew()

In [ ]:
# Remove selling_price column
column_skewed = preprocessed_df.columns.drop('selling_price')

# Compute skewness for each column
skewness = preprocessed_df.drop(columns=['selling_price']).skew()
print(skewness)

plt.figure(figsize=(15, 7))
sns.barplot(x=column_skewed, y=skewness, hue=column_skewed, palette='coolwarm')

plt.title('Skewness of Features in Car Dataset', fontsize=16, pad=20)

plt.xlabel('Features', fontsize=12)
plt.ylabel('Skewness', fontsize=12)

plt.axhline(y=0, color='black', linestyle='--', linewidth=1)

plt.xticks(rotation=45, ha='right', fontsize=10)

plt.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
preprocessed_df.head()

In [ ]:
preprocessed_df.info()

## Save the preprocessed data and label encoder

In [ ]:

# Đường dẫn thư mục lưu dữ liệu
processed_data_dir = "processed_data"
saved_processed_data_path = os.path.join(processed_data_dir, "preprocessed_dataset.csv")
saved_label_encoders_path = os.path.join(processed_data_dir, "label_encoders.sav")

# Tạo thư mục nếu chưa tồn tại
os.makedirs(processed_data_dir, exist_ok=True)

# Ghi DataFrame ra CSV
preprocessed_df.to_csv(saved_processed_data_path, index=False)

# Lưu label encoders
joblib.dump(label_encoders, saved_label_encoders_path)

In [ ]:
df_temp = pd.read_csv(saved_processed_data_path)
threshold = 3  # Giá trị ngưỡng để so sánh

# Đếm số lượng giá trị bằng 0 theo từng cột
zero_counts = (df_temp == 0).sum()

# Đếm số lượng giá trị nhỏ hơn ngưỡng threshold theo từng cột
below_threshold_counts = (df_temp < threshold).sum()

# Hiển thị kết quả
print("Count of zero values per column:\n", zero_counts)
print("\nCount of values below threshold per column:\n", below_threshold_counts)

## Clustering

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


# Drop non-numeric columns (if necessary)
df_numeric = preprocessed_df.drop(columns=["name"])  # 'name' is a categorical feature

# Feature scaling (K-Means is sensitive to different scales)
cluestering_scaler = StandardScaler()
df_scaled = cluestering_scaler.fit_transform(df_numeric)



In [ ]:
# Find the optimal K using the Elbow method
inertia = []
K_range = range(1, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

# Plot the Elbow method result
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertia, marker='o', linestyle='--')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method to Determine Optimal K')
plt.show()


In [ ]:
# Choose the optimal K (e.g., based on the elbow method)
optimal_k = 3  # Change this based on the elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_numeric["Cluster"] = kmeans.fit_predict(df_scaled)

# Add cluster labels to original dataframe
preprocessed_df["Cluster"] = df_numeric["Cluster"]




In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=preprocessed_df["mileage"], y=preprocessed_df["selling_price"], hue=preprocessed_df["Cluster"], palette="viridis", s=80)
plt.xlabel("Mileage (km/l)")
plt.ylabel("Selling Price (Lakhs)")
plt.title(f"K-Means Clustering with {optimal_k} Clusters")
plt.legend(title="Cluster")
plt.show()


In [ ]:
# Compute the average values for each cluster
cluster_summary = df_numeric.groupby("Cluster").mean()
cluster_summary


## Modeling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Load data
preprocessed_df = pd.read_csv(saved_processed_data_path)


In [ ]:
# Split data into input and label data
X = preprocessed_df.drop(columns = ['selling_price'])
Y = preprocessed_df['selling_price']
print(f'size of input data {X.shape}')
print(f'size of input data {Y.shape}')

In [ ]:
# Split data into train and test data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.12, random_state = 42)
print(f'x train size {x_train.shape}, x test size {x_test.shape}')
print(f'y train size {y_train.shape}, y test size {y_test.shape}')

In [ ]:
# Normalize data (fit on train, transform on test)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)



In [ ]:
# Create all models
models = {
    'Linear Regression': LinearRegression(), 
    'Ridge': Ridge(),
    'Lasso': Lasso(), 
    'Decision Tree': DecisionTreeRegressor(),
    'KNN': KNeighborsRegressor(),
    'Random Forest': RandomForestRegressor()
}

In [ ]:
results = []
best_model = None
best_model_name = None
best_mse = float('inf')

# Iterate over each model in the models dictionary
for name, model in models.items():
    # Fit the model on the scaled training data
    model.fit(x_train_scaled, y_train)
    
    # Predict on training and testing datasets
    train_pred = model.predict(x_train_scaled)
    test_pred = model.predict(x_test_scaled)
    
    # Calculate evaluation metrics for training data
    train_mse = mean_squared_error(y_train, train_pred)
    train_mae = mean_absolute_error(y_train, train_pred)
    train_r2 = r2_score(y_train, train_pred)
    
    # Calculate evaluation metrics for testing data
    test_mse = mean_squared_error(y_test, test_pred)
    test_mae = mean_absolute_error(y_test, test_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    # Append all metrics to results list
    results.append({
        'Model': name,
        'Train MSE': train_mse,
        'Test MSE': test_mse,
        'Train MAE': train_mae,
        'Test MAE': test_mae,
        'Train R^2': train_r2,
        'Test R^2': test_r2
    })
    
    # Update the best model based on Test MSE
    if test_mse < best_mse:
        best_mse = test_mse
        best_model = model
        best_model_name = name

# Create a DataFrame to display results
df_results = pd.DataFrame(results)
df_results

In [ ]:
# Define the metrics and their corresponding DataFrame columns
metrics = [
    ('MSE', 'Train MSE', 'Test MSE'),
    ('MAE', 'Train MAE', 'Test MAE'),
    ('R^2', 'Train R^2', 'Test R^2')
]

# Create subplots: one row per metric
fig, axes = plt.subplots(nrows=len(metrics), ncols=1, figsize=(10, 18))

# Loop over each metric to plot its bar chart
for ax, (metric_name, train_col, test_col) in zip(axes, metrics):
    # Set Model as index and select the current metric columns
    df_plot = df_results.set_index('Model')[[train_col, test_col]]
    
    # Plot the bar chart for the current metric
    df_plot.plot(kind='bar', ax=ax, rot=45, title=f"{metric_name} Comparison")
    
    # Add data labels on top of each bar
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', padding=3)
    
    # Set the y-axis label
    ax.set_ylabel(metric_name)

plt.tight_layout()
plt.show()


In [ ]:
joblib.dump(best_model, "best_model.pkl")
print(f"Saved {best_model_name} to best_model.pkl")

In [ ]:
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_names = X.columns
    
    # Sắp xếp giảm dần theo độ quan trọng
    sorted_idx = np.argsort(importances)[::-1]
    sorted_importances = importances[sorted_idx]
    sorted_features = feature_names[sorted_idx]
    
    plt.figure(figsize=(8, 6))
    plt.barh(sorted_features, sorted_importances, color='skyblue')
    plt.gca().invert_yaxis()  # Để feature cao nhất ở trên cùng
    plt.title(f"Feature Importances - {best_model_name}")
    plt.xlabel("Importance")
    plt.show()
else:
    print(f"The model {best_model_name} does not have the 'feature_importances_' attribute")


## Load the best model

In [ ]:
# Load model
loaded_model = joblib.load("best_model.pkl")

# Predict
predictions = loaded_model.predict(x_test_scaled)

zero_count = np.sum(x_test_scaled == 0)
zero_count_2 = np.sum(predictions == 0)
zero_count_3 = (y_test < 3).sum()


In [ ]:
print(zero_count_3)

In [ ]:
print(type(y_test))

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, alpha=0.6)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title(f"Actual vs Predicted ({best_model_name})")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.show()


In [ ]:
# Define threshold
y_threshold = max(y_test) / 3

mask = y_test <= y_threshold
filtered_y_test = y_test[mask]
filtered_predictions = predictions[mask]

plt.figure(figsize=(8, 6))
plt.scatter(filtered_y_test, filtered_predictions, alpha=0.6)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title(f"Actual vs Predicted (Filtered) - {best_model_name}")
plt.plot([min(filtered_y_test), max(filtered_y_test)], 
         [min(filtered_y_test), max(filtered_y_test)], color='red')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label="Actual", marker='o', linestyle='dashed')
plt.plot(predictions, label="Predicted", marker='x', linestyle='dotted')
plt.xlabel("Data Points")
plt.ylabel("Price")
plt.title(f"Actual vs Predicted Prices ({best_model_name})")
plt.legend()
plt.show()
